## Where am I?

In [ ]:
getwd()

## Clear global environment

In [ ]:
rm(list = ls())

## Setup *sparklyr*

In [ ]:
setupLibrary <- function(libraryName){
  if (!require(libraryName, character.only = TRUE)){
    install.packages(libraryName, dep = TRUE)
    if (!require(libraryName, character.only = TRUE)){
      print('Package not found')
    }
  } else {
    print('Package is loaded')
  }
}

setupLibrary('sparklyr')

In [ ]:
sc <- spark_connect(master = 'yarn', 
                    config = list('spark.driver.memory'='8G',
                                  'spark.executor.instances'=4,
                                  'spark.executor.cores'=8,
                                  'spark.executor.memory'='8G')
                    )

## Airline Traffic Delay



In [ ]:
airline_tlb <- spark_read_csv(sc, name = 'airline_data',
                              path = '/repository/airlines/data/',
                              delimiter = ',')

In [ ]:
sdf_dim(airline_tlb)

In [ ]:
object.size(airline_tlb)

In [ ]:
airline_tlb

**Column Descriptions:**

1. Year:	1987-2008
2.	Month:	1-12
3.	DayofMonth:	1-31
4.	DayOfWeek:	1 (Monday) - 7 (Sunday)
5.	DepTime:	actual departure time (local, hhmm)
6.	CRSDepTime:	scheduled departure time (local, hhmm)
7.	ArrTime:	actual arrival time (local, hhmm)
8.	CRSArrTime:	scheduled arrival time (local, hhmm)
9.	UniqueCarrier:	unique carrier code
10.	FlightNum:	flight number
11.	TailNum:	plane tail number
12.	ActualElapsedTime:	in minutes
13.	CRSElapsedTime:	in minutes
14.	AirTime:	in minutes
15.	ArrDelay:	arrival delay, in minutes
16.	DepDelay:	departure delay, in minutes
17.	Origin:	origin IATA airport code
18.	Dest:	destination IATA airport code
19.	Distance:	in miles
20.	TaxiIn:	taxi in time, in minutes
21.	TaxiOut:	taxi out time in minutes
22.	Cancelled:	was the flight cancelled?
23.	CancellationCode:	reason for cancellation (A = carrier, B = weather, C = NAS, D = security)
24.	Diverted:	1 = yes, 0 = no
25.	CarrierDelay:	in minutes
26.	WeatherDelay:	in minutes
27.	NASDelay:	in minutes
28.	SecurityDelay:	in minutes
29.	LateAircraftDelay:	in minutes

**Do planes with a delayed departure fly with a faster average speed to make up for the delay?**

- Group data by individual categories (time of day, day of week, time of year) and calculate average delay in each category
- How to quickly organize hierarhical groupping?
- Are there bad data? (*Hint: Why are ArrDelay and DepDelay char but not int?*)

In [ ]:
setupLibrary('dplyr')

We check for bad data first

In [ ]:
tmp_tlb <- airline_tlb %>% 
  distinct(DepDelay)

In [ ]:
sdf_dim(tmp_tlb)

This is small enough to bring back for detailed observations

In [ ]:
unique_depdelay <- tmp_tlb %>% collect()

In [ ]:
x <- as.numeric(unique_depdelay[[1]])
subset(x, is.na(x))

It seems that there is no non-numeric when we attempt to convert `unique_depdelay`

**Challenge: Check whether bad data exists for `arrdelay`**

**Average delay during different day of week**

In [ ]:
delay_timeofday <- airline_tlb %>% 
  mutate(numeric_depdelay = as.numeric(DepDelay)) %>%
  mutate(numeric_arrdelay = as.numeric(ArrDelay)) %>%
  group_by(DayOfWeek) %>%
  summarize(avg_depdelay = mean(numeric_depdelay),
           avg_arrdelay = mean(numeric_arrdelay)) %>%
  collect            

In [ ]:
delay_timeofday

**Challenge: Check average delay during month of year**

**Do older planes suffer more delays?**

In [ ]:
planes_tlb <- spark_read_csv(sc, name = 'plane_data',
                              path = '/repository/airlines/metadata/plane-data.csv',
                              delimiter = ',')

In [ ]:
planes_tlb

In [ ]:
sdf_dim(planes_tlb)

In [ ]:
clean_planes_tlb <- planes_tlb %>%
  filter(!is.na(year))
sdf_dim(clean_planes_tlb)

In [ ]:
clean_planes_tlb

We need to perform the merge early

In [ ]:
planes_year <- airline_tlb %>%
  mutate(numeric_depdelay = as.numeric(DepDelay)) %>%
  mutate(numeric_arrdelay = as.numeric(ArrDelay)) %>%
  select(TailNum, numeric_depdelay, numeric_arrdelay) %>%
  inner_join(clean_planes_tlb, by = c('TailNum' = 'tailnum')) %>%
  select(year, numeric_depdelay, numeric_arrdelay)

In [ ]:
sdf_dim(planes_year)

In [ ]:
planes_year

In [ ]:
avg_delay_planes_year <- planes_year %>%
  group_by(year) %>%
  summarize(avg_depdelay = mean(numeric_depdelay),
           avg_arrdelay = mean(numeric_arrdelay)) %>%
  filter(year != 'None') %>%
  collect

In [ ]:
summary(avg_delay_planes_year)

In [ ]:
setupLibrary('ggplot2')  
setupLibrary('reshape2')

In [ ]:
avg_delay_planes_year <- melt(avg_delay_planes_year, id.vars="year", variable.name = 'delay_type', value.name = 'delay_time')
head(avg_delay_planes_year)

In [ ]:
ggplot(data = avg_delay_planes_year, 
       aes(x = year, y = delay_time, group = delay_type, colour = delay_type)) +
    geom_line() + theme_bw() +
    theme(axis.text.x = element_text(colour="grey20", size=8, angle=90, hjust=.5, vjust=.5),
                        axis.text.y = element_text(colour="grey20", size=12),
          text=element_text(size=16, family="Arial"))

In [ ]:
spark_disconnect(sc)